In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
path = "/content/drive/My Drive/Aspect-based Sentiment Analysis/"

In [3]:
with open(path+'VLSP2018/VLSP2018-SA-Restaurant-train.txt','r') as file:
  vlsp_res_train = file.readlines()
with open(path+'VLSP2018/VLSP2018-SA-Restaurant-dev.txt','r') as file:
  vlsp_res_dev = file.readlines()
with open(path+'VLSP2018/VLSP2018-SA-Restaurant-test.txt','r') as file:
  vlsp_res_test = file.readlines()

In [4]:
def get_data(data):
  get = 1
  review = []
  label = []
  for text in data:
    if get == 2:
      review.append(text)
      get+=1
    elif get == 3:
      label.append(text)
      get+=1
    elif get == 4:
      get=1
    else: get+=1
  return review,label

In [23]:
vlsp_train_review,vlsp_train_label = get_data(vlsp_res_train)
vlsp_dev_review,vlsp_dev_label = get_data(vlsp_res_dev)
vlsp_test_review,vlsp_test_label = get_data(vlsp_res_test)

In [6]:
Aspect = ['RESTAURANT#GENERAL','RESTAURANT#PRICE','RESTAURANT#MISCELLANEOUS','FOOD#QUALITY','FOOD#STYLE&OPTIONS',
          'FOOD#PRICE','DRINKS#QUALITY','DRINKS#STYLE&OPTIONS','DRINKS#PRICE','SERVICE#GENERAL','AMBIENCE#GENERAL','LOCATION#GENERAL']
Pos = [1,0,0]
Neg = [0,1,0]
Neu = [0,0,1]
Not = [0,0,0]
sentiments = ['positive','negative','neutral']

In [7]:
def get_aspect_label(label):
  aspect_label = []
  for i in label:
    aspect_text = []
    for j in Aspect:
      if j in i:
        aspect_text.append(1)
      if j not in i:
        aspect_text.append(0)
    aspect_label.append(aspect_text)
 
  return aspect_label

In [24]:
aspect_label_train = get_aspect_label(vlsp_train_label)
aspect_label_dev = get_aspect_label(vlsp_dev_label)
aspect_label_test = get_aspect_label(vlsp_test_label)

In [9]:
def get_sentiment_label(label):
  sentiment_label = []
  for i in label:
    temp_list = [0 for i in range(len(Aspect)*3)]
    for k in i.split('}, '):
      sentiment_text = []
      for j in Aspect:
        if j in k:
          if sentiments[0] in k:
            sentiment_text+=Pos
          if sentiments[1] in k:
            sentiment_text+=Neg
          if sentiments[2] in k:
            sentiment_text+=Neu
        if j not in k:
          sentiment_text+=Not
      temp_list = [temp_list[index] + sentiment_text[index] for index in range(len(Aspect)*3)]
    sentiment_label.append(temp_list)
  return sentiment_label

In [25]:
sentiment_label_train = get_sentiment_label(vlsp_train_label)
sentiment_label_dev =  get_sentiment_label(vlsp_dev_label)
sentiment_label_test =  get_sentiment_label(vlsp_test_label)

In [11]:
#@title install vncorenlp
!pip3 install vncorenlp
!mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/ 
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/

     |████████████████████████████████| 2.7MB 2.8MB/s 
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-cp36-none-any.whl size=2645934 sha256=fd9230bddbe06a9ea0d70c6d5c48e5ae17c10796c26204bd00ff9fe200cd7e5f
  Stored in directory: /root/.cache/pip/wheels/09/54/8b/043667de6091d06a381d7745f44174504a9a4a56ecc9380c54
Successfully built vncorenlp
--2020-10-06 05:02:04--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27412575 (26M) [application/octet-stream]
Saving to: ‘VnCoreNLP-1.1.1.jar’

VnCoreNLP-1.1.1.jar 100%[===================>]  26.14M  38.0MB/s    in 0.7s    

2020-10-06 05:02:05 (38.0 MB/s) - ‘VnCoreNLP-1.1.1.jar’ saved [27412575/27412575]

--2020-10-06 05:02:05--  http

In [12]:
from vncorenlp import VnCoreNLP
tokenizer = VnCoreNLP("/content/vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 

In [13]:
import pickle 
replace_list = pickle.load(open(path+'replace_list.pkl','rb'))

In [14]:
pickle.dump(replace_list,open(path+'replace_list.pkl','wb'))

In [15]:
import re
from gensim.utils import simple_preprocess
import pandas as pd
from nltk import flatten

In [16]:
VN_CHARS_LOWER = u'ạảãàáâậầấẩẫăắằặẳẵóòọõỏôộổỗồốơờớợởỡéèẻẹẽêếềệểễúùụủũưựữửừứíìịỉĩýỳỷỵỹđð'
VN_CHARS_UPPER = u'ẠẢÃÀÁÂẬẦẤẨẪĂẮẰẶẲẴÓÒỌÕỎÔỘỔỖỒỐƠỜỚỢỞỠÉÈẺẸẼÊẾỀỆỂỄÚÙỤỦŨƯỰỮỬỪỨÍÌỊỈĨÝỲỶỴỸÐĐ'
VN_CHARS = VN_CHARS_LOWER + VN_CHARS_UPPER
def no_marks(s):
    __INTAB = [ch for ch in VN_CHARS]
    __OUTTAB = "a"*17 + "o"*17 + "e"*11 + "u"*11 + "i"*5 + "y"*5 + "d"*2
    __OUTTAB += "A"*17 + "O"*17 + "E"*11 + "U"*11 + "I"*5 + "Y"*5 + "D"*2
    __r = re.compile("|".join(__INTAB))
    __replaces_dict = dict(zip(__INTAB, __OUTTAB))
    result = __r.sub(lambda m: __replaces_dict[m.group(0)], s)
    return result

In [21]:
def preprocess(data):
    token = []
    token_nomarks = []
    for text in data:
        check = re.search(r'([a-z])\1+',text)
        if check:
          if len(check.group())>2:
            text = re.sub(r'([a-z])\1+', lambda m: m.group(1), text, flags=re.IGNORECASE) #remove các ký tự kéo dài như hayyy, ngonnnn...

        text = text.strip() #loại dấu cách đầu câu
      
        for k, v in replace_list.items():       #replace các từ có trong replace_list
          text = text.replace(k, v)       
        text = re.sub('[!”"#$%&’()•/:;<=>-?@[\]^`{|}~+*_-]', ' ', text) #special character
        text = re.sub('\d+k', '', text) #300k, 200k 
        text = re.sub('\d+', '', text) #number
        text = re.sub('(https|http)?:\/(\w|\.|\/|\?|\=|\&|\%)*\b', ' ', text) #web address
        text = re.sub('www\.\S+\.com', ' ', text) #web address
        text = re.sub('@\S+', ' ', text) #user mention
        text = re.sub('[0-9]k', ' ', text)
        emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
        text = re.sub(emoji_pattern, '', text)
        
        text = ' '.join(i for i in flatten(tokenizer.tokenize(text)))             #gán từ ghép
        text_nomarks = token_nomarks.append(no_marks(text))

        token.append(text)
    return token, token_nomarks

In [26]:
vlsp_train_review, vlsp_train_review1  = preprocess(vlsp_train_review)
vlsp_train_review += vlsp_train_review1
sentiment_label_train += sentiment_label_train
aspect_label_train+=aspect_label_train
vlsp_dev_review, vlsp_dev_review1 = preprocess(vlsp_dev_review)
vlsp_dev_review += vlsp_dev_review1
sentiment_label_dev += sentiment_label_dev
aspect_label_dev += aspect_label_dev
vlsp_test_review, vlsp_test_review1 = preprocess(vlsp_test_review)
vlsp_test_review += vlsp_test_review1
sentiment_label_test += sentiment_label_test
aspect_label_test += aspect_label_test

In [27]:
import pickle
df = pd.DataFrame(list(zip(vlsp_train_review)),columns=['review'])
df1 = pd.DataFrame(list(zip(vlsp_dev_review)),columns=['review'])
df2 = pd.DataFrame(list(zip(vlsp_test_review)),columns=['review'])
df.to_csv(path+'review_train2.csv')
df1.to_csv(path+'review_dev2.csv')
df2.to_csv(path+'review_test2.csv')

with open(path+'aspect_train.pkl', 'wb') as file:
  pickle.dump(aspect_label_train,file)
with open(path+'aspect_dev.pkl', 'wb') as file:
  pickle.dump(aspect_label_dev,file)
with open(path+'aspect_test.pkl', 'wb') as file:
  pickle.dump(aspect_label_test,file)

with open(path+'sentiment_train.pkl', 'wb') as file:
  pickle.dump(sentiment_label_train,file)
with open(path+'sentiment_dev.pkl', 'wb') as file:
  pickle.dump(sentiment_label_dev,file)
with open(path+'sentiment_test.pkl', 'wb') as file:
  pickle.dump(sentiment_label_test,file)